## Задание 1 (декоратор)
Есть класс ***DeviceConnector***, позволяющий общаться с умной лампочкой в локальной сети. 
### Задача:
Вам необходимо реализовать декоратор ***auto_reconnect*** осуществляющий повторную попытку отправки сообщения на девайс при потере соединения.
В случае если отправка не удалась, должно вызваться исключение ***DeviceDisconnected***.  
Если же соединение с устройством не было ранее установленно(флаг ***_is_connected*** равен **False**), должно вызваться исключение ***DeviceNotConnected***.

In [ ]:
import socket
from time import sleep
from typing import Callable, Tuple
import struct

class DeviceNotConnected(Exception):
    def __init__(self, ip, port):
        self._ip = ip
        self._port = port

    def __str__(self):
        return f'Device {self._ip}:{self._port} is not connected, call connect() first.'
    
class DeviceDisconnected(Exception):
    def __init__(self, ip, port, tries):
        self._ip = ip
        self._port = port
        self._tries = tries

    def __str__(self):
        return f'Device {self._ip}:{self._port} is disconnected. Tried {self._tries} times.'

def auto_reconnect(tries: int = None, delay: int = 1) -> Callable:
    """
    Decorator for reconnecting to device. If device is not connected then raise DeviceNotConnected exception.
    @param tries: number of tries, if None then infinite
    @param delay: delay between tries in seconds
    """
    #TODO: Here is your code
    pass

class DeviceConnector:
    def __init__(self, ip, port):
        self._ip = ip
        self._port = port
        self._is_connected = False
        self._socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self._socket.settimeout(1)

    @property
    def is_connected(self) -> bool:
        return self._is_connected
    
    def connect(self):
        self._socket.connect((self._ip, self._port))
        self._is_connected = True

    def disconnect(self):
        self._is_connected = False
        self._socket.close()

    @auto_reconnect(tries=3, delay=1)
    def turn_on(self):
        self._socket.send('ON'.encode())

    @auto_reconnect(tries=3, delay=1)
    def turn_off(self):
        self._socket.send('OFF'.encode())
        

    @auto_reconnect(tries=3, delay=1)
    def set_color(self, color: hex):
        self._socket.send(f'COLOR {color}'.encode())

    @auto_reconnect(tries=3, delay=1)
    def set_brightness(self, color: Tuple[float, float, float]):
        message = struct.pack('!3f', *color)
        self._socket.send(message)


## Задание 2 (итератор)
Общение с устроством происходит по протоколу, в котором каждый бит передается по очереди с опредленной задержкой, указанной в списке ***delays***.
Есть класс ***ToBinary***, позволяющий получить двоичное представление числа неограниченной длины в виде строки.  
### Задача:  
Вам необходимо реализовать недостающие магические методы класса ***ToBinary***, чтобы можно было итерироваться по битам числа.  
Обратите внимание, что в данном случае используется именно итератор, а не генератор ввиду необходимости получения длины последовательности заранее.

In [ ]:
from typing import Literal

class ToBinary:
    def __init__(self, number: int, order: Literal["big", "little"] = "big"):
        """
        Класс, который переводит число в двоичную систему счисления
        @param number: число, которое нужно перевести в двоичную систему счисления (positive int)
        @param order: порядок перевода: big-endian или little-endian
        """
        assert number >= 0, "Number must be positive"
        assert order in ("big", "little"), "Order must be 'big' or 'little'"
        self._number = number
        self._order = order
        self._binary_str = self._binarize()

    def _byte_to_binary(self, byte: int) -> str:
        """
        Переводит байт в двоичную систему счисления
        @param byte: байт, который нужно перевести в двоичную систему счисления (positive int)
        @return: строка, представляющая двоичное число
        """
        assert 0 <= byte <= 255, "Byte must be in range [0, 255]"
        binary = ""
        for i in range(8):
            binary = str(byte % 2) + binary
            byte //= 2
        return binary if  self._order == "big" else binary[::-1]
    
    def _binarize(self) -> str:
        """
        Переводит число в двоичную систему счисления
        @return: строка, представляющая двоичное число
        """
        #TODO: Here is your code
        pass
    
    #TODO: Here is your code

from time import sleep

def test_2():
    instance = ToBinary(int(input("Enter positive number: ")), order="big")
    iterator = None
    #TODO: Here is your code 
    # iterator = None
    bits_count = len(iterator)
    delays = [i ** 2 for i in range(bits_count)]
    for bit, delay in zip(iterator, delays):
        #TODO: Here is your code
        sleep(delay)
    
test_2()

## Задание 3 (контекстный менеджер)
Есть класс ***DatasetRecorder***, позволяющий общаться c собирающим датасет с некоторого робота сервисом.  
У сервиса есть эндпоинт для начала записи датасета, а также эндпоинт для окончания записи датасета.
### Задача:
Вам необходимо реализовать контекстный менеджер ***record_dataset*** класса ***DatasetRecorderContextManager***, который будет оборачивать блок кода, в котором происходит запись датасета.  
В случае, если сервис не ответил (вызвалось исключение ***ConnectionError***), необходимо повторить попытку отправки запроса ***max_tries*** раз.  
Если после ***max_tries*** попыток сервис так и не ответил, необходимо выполнить блок кода, обернутый в контекстный менеджер, написав в лог сообщение ```can't connect to "dataset recorder" service```.  
Используйте модуль ***logging*** для логирования и модуль ***contextlib*** для реализации контекстного менеджера.

In [1]:
from random import random
import time
import logging
from typing import Literal
from contextlib import contextmanager

def post_request_mock(service_address: str, command: Literal["start", "stop"], parameters: dict) -> str:
    logger = logging.getLogger("dataset_logger")
    if command == "stop":
        logger.info("stopped recording")
        return "OK"
    if random() < 0.5:
        raise ConnectionError("Connection to server failed")
    else:
        logger.info(f'calling service {service_address} with parameters {parameters} for starting recording')
        return "OK"
    
class DatasetRecorderContextManager:
    def __init__(self, service_address: str, timestamp: str, robot_id: str, robot_type: str, max_tries: int = 3):
        self._logger = logging.getLogger("dataset_logger")
        self._service_address = service_address
        self._data = {
            "timestamp": timestamp,
            "robot_id": robot_id,
            "robot_type": robot_type
        }
    
    #TODO: Here is your code

    
class Robot:
    def __init__(self):
        self._logger = logging.getLogger("robot")
        pass
    
    @property
    def logger(self):
        return self._logger
    
    @logger.setter
    def logger(self, logger):
        self._logger = logger

    def move_forward_for(self, distance: float):
        self._logger.info("moving forward")
        sleep(distance)
        self._logger.info(f"moved forward on {distance} meters")

    def move_backward_for(self, distance: float):
        self._logger.info("moving backward")
        sleep(distance)
        self._logger.info(f"moved backward on {distance} meters")

    def turn_left_for(self, angle: float):
        self._logger.info("turning left")
        sleep(angle // 20)
        self._logger.info(f"turned left on {angle} degrees")

    def turn_right_for(self, angle: float):
        self._logger.info("turning right")
        sleep(angle // 20)
        self._logger.info(f"turned right on {angle} degrees")

    def stop(self):
        self._logger.info("stopped")

        
def test_3():
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger("main_thread")
    robot = Robot()
    dataset_data = {
        "service_address": "http://localhost:8080",
        "timestamp": time.time(),
        "robot_id": "123",
        "robot_type": "rover"
    }
    with DatasetRecorderContextManager(**dataset_data) as response:
        logger.info(f"is record started: {response}")
        robot.move_forward_for(2)
        robot.move_backward_for(4)
        robot.turn_left_for(90)
        robot.turn_right_for(90)
        robot.stop()
    logger.info("finished")


## Задание 4 (генератор)

### Задача:
Необходимо считывать данные из датасета (строка ***dataset***) и "налету" преобразовывать их в словарь, содержащий информацию о состоянии робота в момент времени ***timestamp***.  
Ожидается, что в словаре будут ключи: ***line_index***, ***timestamp***, ***joint_poses***, ***joint_velocities***, ***joint_torques***, ***end_effector_linear_velocity***, ***end_effector_linear_torque***.  
Строчки в датасете разделены символом переноса строки ```n``` и содержат следующие данные:  
```timestamp,, joint_poses,, joint_velocities,, joint_torques,, end_effector_linear_velocity,, end_effector_linear_torque```  
Реализуйте генератор ***parse_dataset***, который будет считывать данные из датасета и возвращать словарь с данными о состоянии робота.


In [ ]:
from typing import Generator, Tuple, List

# Dataset format: line_index,
# timestamp,, joint_poses,, joint_velocities,, joint_torques,, end_effector_linear_velocity,, end_effector_linear_torque
dataset = \
"""0,, [0.0, 0.0, 0.0],, [0.0, 0.0, 0.0],, [0.0, 0.0, 0.0],, 0,, 0
0.1,, [0.0, 0.05, 0.5],, [0.0, 0.0, 0.1],, [0.0, 0.0, 0.1],, 0,, 0
0.2,, [0.0, 0.1, 1.0],, [0.4, 0.0, 0.0],, [0.6, 0.0, 0.0],, 0,, 1
0.3,, [0.0, 0.15, 1.5],, [0.0, 0.0, 0.0],, [0.0, 0.0, 0.0],, 0,, 0
0.4,, [0.0, 0.2, 2.0],, [0.0, 0.0, 0.0],, [0.0, 0.0, 0.0],, 0,, 0
0.5,, [0.5, 0.25, 2.5],, [0.0, 0.0, 0.0],, [0.0, 0.0, 0.0],, 0,, 0
0.6,, [0.5, 0.25, 2.5],, [0.0, 0.0, 0.0],, [0.0, 0.0, 0.0],, 0,, 0"""

def parse_dataset(dataset: str) -> Generator[dict, None, None]:
    """
    Функция, которая парсит датасет и возвращает генератор
    @param dataset: строка, представляющая датасет
    @return: генератор, который возвращает словари вида:
        {
            "line_index": int,
            "timestamp": float,
            "joint_poses": List[float],
            "joint_velocities": List[float],
            "joint_torques": List[float],
            "end_effector_linear_velocity": float,
            "end_effector_linear_torque": float }
    """
    #TODO: Here is your code
    pass
            

def test_4():
    for data in parse_dataset(dataset):
        data_str = ""
        data_str += "\n\t".join([f"{key}: {value} ({str(type(value))})" for key, value in data.items() if key != "line_index"])
        print(f"line {data['line_index']}\n\t{data_str}\n")
        
test_4()